#### KoAlpaca-Polyglat-12.8B 8bit 양자화 모델 추론 
```
1. excute: github: https://github.com/Beomi/KoAlpaca/tree/main/webui/app.py 
2. gqpt: github: https://github.com/qwopqwop200/GPTQ-for-KoAlpaca/blob/main/quant_with_alpaca.py
3. parameter data type: touch.float16
4. qutanization: 8 bit quant
5. processor: cpu
6. write type: pytorch bin
7. transformers: AutoModelForCausalLM, pipeline
```

#### requiment modules
```
pip install autotrain-advanced
pip install transformers=4.38.2
pip install bitsandbytes
pip install accelerate
pip install sentencepiece
pip install protobuf
```

In [1]:
import torch
from transformers import AutoModelForCausalLM, pipeline, BitsAndBytesConfig

In [2]:
# GPU 확인 함수
def isUsingGpu():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"GPU is available. Using GPU.{device}")
        return True
    else:
        device = torch.device("cpu")
        print(f"GPU is not available. Using CPU.{device}")
        return False
    
# model loaded porcessor
def is_model_on_gpu(model):
    return next(model.parameters()).is_cuda    

In [3]:
if torch.cuda.is_available():
    torch.cuda.empty_cache() # 사용되지 않는 캐시된 메모리 해제
    print("GPU cache is emptied.")

GPU cache is emptied.


In [ ]:
max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)}GB'
print(f"cuda max memory: {max_memory}")

In [5]:
# KoAlpaca-ployglot 12.8B 8BIT 양자화 모델
# 파일형식: PyTouch 모델 저장 방식 (확장자: bin)
# 파일크기: 9.5G, 3.1G 2개
model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bhf-8bit"

# 방법1) 최적 응답속도: 1분 5.6초 저품질
'''
model = AutoModelForCausalLM.from_pretrained(
    model_id,    
    device_map="auto",       
    revision="8bit",
    low_cpu_mem_usage=True,    
)
'''

# 방법2) 응답속도: 5분 44.2초 고품질
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    revision="8bit",
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# 로딩 모델 위치
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
# 전체 파라메터 확인
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_parameters_in_billion = total_parameters / 1e9
print(f"Total trainable parameters: {total_parameters}")
print(f"Total trainable parameters: {total_parameters_in_billion} billion")

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
# 장치 확인   
using_cuda = isUsingGpu()
on_gpu_model = is_model_on_gpu(model)

print(f"GPU Using: {using_cuda}, Model GPU Loaded: {on_gpu_model}")
# GPU 가 사용가능하고, 모델이 CPU에 로딩되어 있다면
# 모델을 GPU 로 이동한다.
if(using_cuda == True and on_gpu_model == False):
    model.to('cuda')

In [ ]:
print(model)

In [ ]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

In [8]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [9]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [10]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

### 질문:
정보화시스템 감리 절차 과정에 대해 설명해줘


In [11]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


.

### 맥락:
정보화 시스템 감리에 대해 2003년 11월에 개정된 감리방법론이 시행되었습니다. 이에 따라  감리절차 등에 대하여 첨부와 같이 게시하오니 업무에 참고하시기 바랍니다.                                                                                                                                                                                                                                     
                                                                
                                                                


### 답변:정보화 시스템 감리에 대한 2003년 11월에 개정된 감리방법론이 시행되었습니다. 이에 따라, 감리 절차 등에 대해 첨부와 같이 게시되었습니다. 

1. 정보시스템 감리 절차 
1) 계획 단계: 프로젝트 시작 전에 정보시스템의 현황과 문제점을 파악하고 분석하여 감리 계획을 수립합니다. 
2) 실행 단계: 정보시스템 구축이 진행되는 동안에는 구축 내용을 검토하며, 예상되는 문제점을 파악합니다. 문제점이 발생하면, 그 문제점을 해결할 수 있는 방안을 강구한 후, 그 결과를 확인하고 해당 시스템의 기능, 구조, 인터페이스 등을 확정합니다. 
3) 평가 단계: 구축이 완료되면, 해당 정보시스템의 기능, 성능, 확장성 등을 평가하며, 분석한 결과를 기반으로 개선 사항을 도출합니다. 

2. 감리 업무 흐름도 
1) 요구 사항 분석: 감리를 요청하는 기업에서 요구사항을 정리하여 전달합니다. 
2) 계획 단계: 전달받은 요구사항을 기반으로 감리 계획을 수립합니다.
3) 감리 진행: 수립된 감리 계획에 따라 감리를 진행합니다.
4) 보고서 작성